In [ ]:
%load_ext autoreload

import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize

import pyOptLE as optle

In [ ]:
%autoreload

In [ ]:
generate = False
n = 100
N = 5000
    
# User needs to provide dt!
dt = 0.1
T = 300

# format data from "real" simulation
filenames = ['data_10ala/decaa_abf_500traj', 'data_10ala/decaa_abmd_500traj', 'data_10ala/decaa_nobias_500traj']

auto_bounds = True
# qmin_user = 11
# qmax_user = 34

n_knots = 30

In [ ]:
# Load or generate data

if generate:
    D = 0.5
    beta = 1
    V = optle.V_harm
    nablaV = optle.nablaV_harm
#     V = optle.V_dblwell
#     nablaV = optle.nablaV_dblwell
    q = optle.overdamped_Langevin(n, N, dt, beta, D, nablaV)
    f = np.zeros_like(q)
else:
    q, f = optle.load_traj(filenames)

In [ ]:
plot_samples = True
plot_traj = True
n_plots = 10

if plot_samples:
    plt.plot([qi.shape[0] for qi in q])
    plt.xlabel('Trajectory index')
    plt.ylabel('N. of pts per trajectory')

if plot_traj:
    plt.figure()
    stride = len(q) // n_plots
    if stride == 0:
        stride = 1
    for qi in q[::stride]:
        plt.plot(qi)
    plt.xlabel('t')
    plt.ylabel('q')

In [ ]:
# Create grid and initial parameter set

epsilon = 1e-10
qmin_data = min([np.min(qi) for qi in q]) - epsilon
qmax_data = max([np.max(qi) for qi in q]) + epsilon

if auto_bounds:
    # Add margin for floating-point precision
    qmin = qmin_data
    qmax = qmax_data
else:
    assert qmin_user <= qmin_data, '%f > %f' % (qmin_user, qmin_data)
    assert qmax_user >= qmax_data, '%f < %f' % (qmax_user, qmax_data)
    qmin = qmin_user
    qmax = qmax_user


knots = np.linspace(qmin, qmax, n_knots)

# Initial guess for the parameters: zero free energy gradient, constant D = 1
initial_params = np.concatenate((np.zeros(n_knots), np.zeros(n_knots) + 1.))

In [ ]:
# Optimize with debiasing

result, hist = optle.optimize_model(initial_params, knots, q, f, dt, T = T)

In [ ]:
# Re-optimize without debiasing

result_nb, hist_nb = optle.optimize_model(initial_params, knots, q, f = None, dt = dt, T = T)

In [ ]:
# Smoother interpolation for plotting
interp_factor = 10

# Fencepost theorem aplied forward and then backward
n_discr_points = interp_factor * (len(knots) - 1) + 1
x = np.linspace(qmin, qmax, n_discr_points)

optimized_params = result.x
optimized_G = optimized_params[:n_knots]
optimized_logD = optimized_params[n_knots:]

optimized_params_nb = result_nb.x
optimized_G_nb = optimized_params_nb[:n_knots]
optimized_logD_nb = optimized_params_nb[n_knots:]


# Integral of linear interpolation

predicted = optle.piecewise_linear_int(x, knots, optimized_G)
predicted_nb = optle.piecewise_linear_int(x, knots, optimized_G_nb)


# Plot the original data and the optimized spline
import matplotlib.pyplot as plt

counts, bins = np.histogram(np.concatenate(q), bins=40)
# FE = -RT * np.log(counts)
# FE -= np.min(FE)
# plt.stairs(FE, bins, label='FE from histogram')
R = 0.00198720425864 # kcal/mol
RT = R * T
plt.stairs(-RT*(np.log(counts)-np.max(np.log(counts))), bins, label='\"Free energy\" from histogram')

plt.plot(x, predicted, label='Optimized free energy (debiased)', marker='.', markevery=interp_factor)
plt.plot(x, predicted_nb, label='Optimized free energy (not debiased)', marker='.', markevery=interp_factor)

plt.xlabel('colvar q')
plt.ylabel('Free energy')

if generate:
    plt.plot(x, V(x) - np.min(V(x)), label='True potential')

# Ad hoc bc we know this is a double well
# plt.plot(x, optle.V_dblwell(x) - np.min(optle.V_dblwell(x)), label='Ideal double well')

plt.legend()

    
    
plt.figure()
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD)), label='Optimized D (debiased)')
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD_nb)), label='Optimized D (not debiased)')

if generate:
    plt.plot([qmin, qmax], [D, D], label='True D')
plt.xlabel('colvar q')
plt.ylabel('Diffusion coefficient')
plt.legend()

plot_gradient = True
if plot_gradient:
    plt.figure()
    predicted_nablaV = np.interp(x, knots, optimized_G)
    plt.scatter(knots, optimized_G, label='Optimized grad G')
    plt.plot(x, predicted_nablaV, label='Optimized gradient')
    if generate:
        plt.plot(x, np.interp(x, knots, nablaV(knots)), label='True gradient')
    plt.xlabel('colvar q')
    plt.ylabel('Diffusion coefficient')
    plt.legend()


In [ ]:
plt.plot(optimized_params)

In [ ]:
traj = optle.preprocess_traj(q, knots)

In [ ]:
a, b, c = traj[0]

In [ ]:
optle.check_gradient(optle.objective_order1_debiased, q, f)